<a href="https://colab.research.google.com/github/Tina-223/DBscan/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

df_train = pd.read_csv('all_drinks.csv')

df_train = df_train.drop(['strCategory','dateModified','strDrinkThumb', 'strGlass', 'strIBA', 'strInstructions', 'strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8',
                          'strMeasure9', 'strMeasure10', 'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15', 'strVideo'], axis=1)
df_train.head()

,Unnamed: 0,strDrink,idDrink,strAlcoholic,strIngredient1,strIngredient10,strIngredient11,strIngredient12,strIngredient13,strIngredient14,strIngredient15,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,strIngredient9
0,0,'57 Chevy with a White License Plate,14029,Alcoholic,Creme de Cacao,NaN,NaN,NaN,NaN,NaN,NaN,Vodka,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1-900-FUK-MEUP,15395,Alcoholic,Absolut Kurant,NaN,NaN,NaN,NaN,NaN,NaN,Grand Marnier,Chambord raspberry liqueur,Midori melon liqueur,Malibu rum,Amaretto,Cranberry juice,Pineapple juice,NaN
2,2,110 in the shade,15423,Alcoholic,Lager,NaN,NaN,NaN,NaN,NaN,NaN,Tequila,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,151 Florida Bushwacker,14588,Alcoholic,Malibu rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,151 proof rum,Dark Creme de Cacao,Cointreau,Milk,Coconut liqueur,Vanilla ice-cream,NaN
4,4,155 Belmont,15346,Alcoholic,Dark rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,Vodka,Orange juice,NaN,NaN,NaN,NaN,NaN
